* Cargar datos dataset

In [1]:
rdd = sc.textFile('datasets/yellow_tripdata_2018-11.csv')

* VendorID
* tpep_pickup_datetime
* tpep_dropoff_datetime
* passenger_count
* trip_distance
* RatecodeID
* store_and_fwd_flag
* PULocationID
* DOLocationID
* payment_type
* fare_amount
* extra
* mta_tax
* tip_amount
* tolls_amount
* improvement_surcharge
* total_amount

In [2]:
print(rdd.take(5))

[u'VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount', u'', u'1,2018-11-01 00:51:36,2018-11-01 00:52:36,1,.00,1,N,145,145,2,2.5,0.5,0.5,0,0,0.3,3.8', u'1,2018-11-01 00:07:47,2018-11-01 00:21:43,1,2.30,1,N,142,164,1,11,0.5,0.5,2.45,0,0.3,14.75', u'1,2018-11-01 00:24:27,2018-11-01 00:34:29,1,1.80,1,N,164,48,2,8.5,0.5,0.5,0,0,0.3,9.8']


In [3]:
def eliminar_cabecera_fila_vacia(line):
    if line.startswith("VendorID") or line.strip() == "":
        return False
    else:
        return True

# Filtra el RDD original 'rdd' con la función 'eliminar_cabecera_fila_vacia'
rdd_filtrado = rdd.filter(eliminar_cabecera_fila_vacia)

# Luego, utiliza 'map' para dividir cada línea en campos
datosSeparados = rdd_filtrado.map(lambda line: line.split(','))
    

# Ahora 'limpiarCabeceraFilaVacia' contendrá el RDD sin las dos primeras líneas





In [4]:
datosSeparados.take(10)

[[u'1',
  u'2018-11-01 00:51:36',
  u'2018-11-01 00:52:36',
  u'1',
  u'.00',
  u'1',
  u'N',
  u'145',
  u'145',
  u'2',
  u'2.5',
  u'0.5',
  u'0.5',
  u'0',
  u'0',
  u'0.3',
  u'3.8'],
 [u'1',
  u'2018-11-01 00:07:47',
  u'2018-11-01 00:21:43',
  u'1',
  u'2.30',
  u'1',
  u'N',
  u'142',
  u'164',
  u'1',
  u'11',
  u'0.5',
  u'0.5',
  u'2.45',
  u'0',
  u'0.3',
  u'14.75'],
 [u'1',
  u'2018-11-01 00:24:27',
  u'2018-11-01 00:34:29',
  u'1',
  u'1.80',
  u'1',
  u'N',
  u'164',
  u'48',
  u'2',
  u'8.5',
  u'0.5',
  u'0.5',
  u'0',
  u'0',
  u'0.3',
  u'9.8'],
 [u'1',
  u'2018-11-01 00:35:27',
  u'2018-11-01 00:47:02',
  u'1',
  u'2.30',
  u'1',
  u'N',
  u'48',
  u'107',
  u'1',
  u'10',
  u'0.5',
  u'0.5',
  u'3.35',
  u'0',
  u'0.3',
  u'14.65'],
 [u'1',
  u'2018-11-01 00:16:46',
  u'2018-11-01 00:22:50',
  u'1',
  u'1.00',
  u'1',
  u'N',
  u'163',
  u'170',
  u'2',
  u'6',
  u'0.5',
  u'0.5',
  u'0',
  u'0',
  u'0.3',
  u'7.3'],
 [u'1',
  u'2018-11-01 00:23:57',
  u'2018-11-0

* A) Contar o número de viaxes entre 00:00 e 01:00 de cada día e dar o total por día (agrupados por día, tódolos días teñen un reconto).


In [5]:
from datetime import datetime

def es_entre_00_01(pickup_str, dropoff_str):
    pickup_hora = datetime.strptime(pickup_str, '%Y-%m-%d %H:%M:%S').time()
    dropoff_hora = datetime.strptime(dropoff_str, '%Y-%m-%d %H:%M:%S').time()
    return pickup_hora.hour == 0 and dropoff_hora.hour == 0

# Filtrar las filas con tiempo de recogida y entrega entre las 00:00 y las 00:59
viajes_00_01 = datosSeparados.filter(lambda row: es_entre_00_01(row[1], row[2]))


# Extraer la fecha (día) de la columna 'tpep_pickup_datetime'
viajes_00_01_fecha = viajes_00_01.map(lambda row: (datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S').date(), 1))

# Agrupar por la fecha y contar el número de viajes en cada grupo
conteo_por_dia = viajes_00_01_fecha.reduceByKey(lambda a, b: a + b)

# 'conteo_por_dia' es un RDD con el conteo de viajes entre 00:00 y 01:00 de cada día

In [6]:
conteo_por_dia.collect()

[(datetime.date(2018, 11, 27), 3980),
 (datetime.date(2018, 11, 15), 5948),
 (datetime.date(2018, 11, 17), 11329),
 (datetime.date(2018, 11, 5), 3124),
 (datetime.date(2018, 11, 9), 8294),
 (datetime.date(2018, 11, 23), 3257),
 (datetime.date(2018, 11, 20), 4797),
 (datetime.date(2018, 11, 24), 6311),
 (datetime.date(2018, 11, 12), 3660),
 (datetime.date(2018, 12, 1), 83),
 (datetime.date(2009, 1, 1), 6),
 (datetime.date(2018, 11, 30), 7971),
 (datetime.date(2018, 11, 2), 7181),
 (datetime.date(2018, 11, 25), 7697),
 (datetime.date(2018, 11, 21), 5751),
 (datetime.date(2018, 11, 3), 10516),
 (datetime.date(2018, 11, 13), 4307),
 (datetime.date(2018, 11, 28), 5416),
 (datetime.date(2018, 11, 6), 3810),
 (datetime.date(2018, 11, 18), 12039),
 (datetime.date(2018, 11, 10), 10748),
 (datetime.date(2018, 11, 1), 7682),
 (datetime.date(2018, 11, 11), 11939),
 (datetime.date(2018, 11, 29), 6011),
 (datetime.date(2018, 11, 19), 2960),
 (datetime.date(2018, 11, 7), 4974),
 (datetime.date(2018, 

* B) Contar o número de viaxes entre 00:00 e 01:00 de cada día e dar o total por mes.

In [7]:

# Extraer el año y mes de cada fecha y contar los viajes por mes
conteo_por_mes = conteo_por_dia.map(lambda row: ((row[0].year, row[0].month), row[1])).reduceByKey(lambda a, b: a + b)

In [8]:
conteo_por_mes.collect()

[((2018, 12), 83), ((2009, 1), 6), ((2018, 11), 198257)]

* C) A media de viaxes ao mes que fai cada conductor.

- Voy a leer de nuevo el archivo para obtener el dataframe de spark del csv

In [9]:
viajes = spark.read.csv('datasets/yellow_tripdata_2018-11.csv', header=True,
                       inferSchema=True)

In [10]:
viajes.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [11]:
viajes.createOrReplaceTempView('viajes')

In [18]:
# Ejecuta la consulta SQL para obtener el conteo de viajes por VendorID y mes
countDF = spark.sql('SELECT VendorID, Month(tpep_pickup_datetime) as mes, count(*) as num_viajes FROM viajes group by VendorID, Month(tpep_pickup_datetime)')

# Muestra el resultado de la consulta anterior
countDF.show()

# Crea una vista temporal para ejecutar consultas SQL en countDF
countDF.createOrReplaceTempView('conteo_viajes')

# Ejecuta una consulta SQL para calcular el promedio de viajes por mes para cada VendorID
viajes_promedio = spark.sql('SELECT VendorID, AVG(num_viajes) as promedio_viajes_por_mes FROM conteo_viajes GROUP BY VendorID')

# Muestra el resultado de la consulta anterior
viajes_promedio.show()

+--------+---+----------+
|VendorID|mes|num_viajes|
+--------+---+----------+
|       4| 11|    130817|
|       1| 11|   3262928|
|       2|  1|        26|
|       2| 11|   4751001|
|       2| 12|       123|
|       2| 10|       269|
+--------+---+----------+

+--------+-----------------------+
|VendorID|promedio_viajes_por_mes|
+--------+-----------------------+
|       1|              3262928.0|
|       4|               130817.0|
|       2|             1187854.75|
+--------+-----------------------+



* D) A media de viaxes ao día que fai cada conductor.

In [19]:
# Calcular la media de viajes al día que hace cada conductor (VendorID)
media_viajes_por_dia = spark.sql("""
    SELECT VendorID, 
           AVG(num_viajes) as promedio_viajes_por_dia
    FROM (
        SELECT VendorID, 
               DATE(tpep_pickup_datetime) as fecha, 
               COUNT(*) as num_viajes
        FROM viajes
        GROUP BY VendorID, DATE(tpep_pickup_datetime)
    ) as viajes_por_dia
    GROUP BY VendorID
""")

# Mostrar el resultado
media_viajes_por_dia.show()

+--------+-----------------------+
|VendorID|promedio_viajes_por_dia|
+--------+-----------------------+
|       1|     108764.26666666666|
|       4|      4360.566666666667|
|       2|             118785.475|
+--------+-----------------------+



* E) Cantos pasaxeiros foron como máximo na primeira semana do mes (nunha viaxe).

In [20]:
# Filtrar viajes en la primera semana del mes y seleccionar el máximo número de pasajeros en una consulta SQL
max_pasajeros_primera_semana = spark.sql("""
    SELECT MAX(passenger_count) as max_pasajeros_primera_semana
    FROM viajes
    WHERE DAY(tpep_pickup_datetime) BETWEEN 1 AND 7
""")

# Mostrar el resultado
max_pasajeros_primera_semana.show()

+----------------------------+
|max_pasajeros_primera_semana|
+----------------------------+
|                           9|
+----------------------------+



* F) Cantos pasaxeiros foron como máximo en todo o mes (nunha viaxe).

In [21]:
max_pasajeros_por_mes = spark.sql("""
    SELECT
        Month(tpep_pickup_datetime) as mes,
        MAX(passenger_count) as max_pasajeros
    FROM viajes
    GROUP BY Month(tpep_pickup_datetime)
""")
max_pasajeros_por_mes.show()

+---+-------------+
|mes|max_pasajeros|
+---+-------------+
| 12|            6|
|  1|            5|
| 10|            6|
| 11|           96|
+---+-------------+



* G) Cantos cartos costou o percorrido máis caro.

In [22]:
max_costo = spark.sql("""
    SELECT
        MAX(total_amount) as costo_mas_caro
    FROM viajes
""")
max_costo.show()

+--------------+
|costo_mas_caro|
+--------------+
|     187437.76|
+--------------+



* H) Cantos cartos costou o percorrido máis barato.

In [23]:
min_costo = spark.sql("""
    SELECT
        MIN(total_amount) as costo_mas_barato
    FROM viajes
""")
min_costo.show()

+----------------+
|costo_mas_barato|
+----------------+
|          -450.3|
+----------------+

